# Proyecto 7: Deploy de sistema de recomendación con Watson

En este proyecto llevaremos a cabo la puesta en producción del modelo entrenado en el proyecto 5. Es decir, lo subirmos la nube de IBM y utilizando llamados a la API de Watson tendremos acceso a él para realizar predicciones.

In [5]:
import warnings
warnings.filterwarnings("ignore")
import sklearn
from sklearn.datasets import load_files
moviedir = r'./dataset/movie_reviews' 
movie_reviews = load_files(moviedir, shuffle=True)

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    movie_reviews.data, movie_reviews.target, test_size = 0.20, stratify=movie_reviews.target, random_state = 12)

In [7]:
from sklearn.externals import joblib
eclf = joblib.load('sentiment.pkl') 

IBM Watson

**1) Cargá** la biblioteca `WatsonMachineLearningAPIClient`

In [16]:
!pip install watson-machine-learning-client --upgrade

Requirement already up-to-date: watson-machine-learning-client in c:\users\matías\anaconda3\lib\site-packages (1.0.371)


In [42]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

**2) Creá** variable con las credenciales que necesita `Watson`. Ellas son: `url, access_key, username, password e instance_id`

In [43]:
wml_credentials={
  "instance_id": "f237cf19-0174-41e4-aa23-1f58a6093eec",
  "password": "b07dc237-329e-47d4-8076-4db5941b7442",
  "url": "https://us-south.ml.cloud.ibm.com",
  "username": "5a31f67f-c9dc-45c9-9d4a-57cafb771e57"
}

**3) Declará** la variable `client` y guardá en ella al objeto `WatsonMachineLearningAPIClient` con las credenciales como parámetro

In [44]:
client = WatsonMachineLearningAPIClient(wml_credentials)

**4) Creá** una variable que guarde las propiedades del modelo. Datos del autor y nombre del proyecto.

In [45]:
metadata = {
            client.repository.ModelMetaNames.NAME: 'Procesamiento_Natural_lenguaje',
            client.repository.ModelMetaNames.FRAMEWORK_NAME: 'scikit-learn',
            client.repository.ModelMetaNames.AUTHOR_NAME: 'Sánchez_Matías',
            client.repository.ModelMetaNames.RUNTIME_NAME: 'python',
            client.repository.ModelMetaNames.RUNTIME_VERSION: '3.5'
}

**5) Hacé** un pipeline que contenga como primer paso a un `TfidfVectorizer` y como segundo paso, al mejor modelo que hayas obtenido en el proyecto 5. **Entrená** con este pipeline.

In [22]:
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer()
pipeline = make_pipeline(vect, eclf)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('tfidfvectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth..., random_state=None))],
         flatten_transform=None, n_jobs=None, voting='hard', weights=None))])

**6) Subí** al modelo a IBM Cloud usando `client.repository.store_model` con los parámetros correctos.

In [46]:
published_model = client.repository.store_model(model=pipeline, 
                                                meta_props=metadata, 
                                                training_data=X_train, 
                                                training_target=y_train)

**7) Obtené** el `uid` del modelo y guardalo en una variable.

In [47]:
models_details = client.repository.list_models()

------------------------------------  ------------------------------  ------------------------  -----------------
GUID                                  NAME                            CREATED                   FRAMEWORK
c7eb98aa-2cd4-4a86-b03b-a936196cc539  Procesamiento_Natural_lenguaje  2019-07-08T05:15:46.800Z  scikit-learn-0.20
4d60208e-ca07-4eec-aaf5-b101407f3540  NLP                             2019-07-08T05:10:16.378Z  scikit-learn-0.20
------------------------------------  ------------------------------  ------------------------  -----------------


In [49]:
guid = client.repository.get_model_uid(published_model)
guid

'c7eb98aa-2cd4-4a86-b03b-a936196cc539'

**8) Cargá** el modelo basado en su `uid` y utilizalo para realizar la predicción sobre el conjunto de test

In [50]:
modelo = client.repository.load(guid)

test_predictions = modelo.predict(X_test)

**9) Mostrar** el `classification_report` obtenido

In [51]:
from sklearn.metrics import classification_report

print(classification_report(y_test, test_predictions))

              precision    recall  f1-score   support

           0       0.82      0.91      0.86       200
           1       0.90      0.80      0.84       200

   micro avg       0.85      0.85      0.85       400
   macro avg       0.86      0.85      0.85       400
weighted avg       0.86      0.85      0.85       400

